In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import os
import sys
import librosa
import numpy as np

current_working_directory = os.getcwd() # get current working directory

# Address of root directory
base_path = current_working_directory
prog_path =     "/content/drive/MyDrive/Music-Genre-Classification-master-20240427T160246Z-001/Music-Genre-Classification-master/Training_set/Progressive_Rock_Songs"
non_prog_path = "/content/drive/MyDrive/Music-Genre-Classification-master-20240427T160246Z-001/Music-Genre-Classification-master/Training_set/Not_Progressive_Rock"

# Returns all audio files(.mp3, .avi, .wav) in the directory : path
def fileList(path) :
    matches = []
    for root, _, filenames in os.walk(path):
        for filename in filenames:
            if filename.endswith(('.mp3', '.wav', '.avi','.flac','.m4a','.ogg')):
                if matches.count(filename) == 0 :
                    matches.append(os.path.join(root, filename))

    return matches


# prog_files contains all prog_rock files
prog_files = fileList(prog_path)


# non_prog_files contains all non prog_rock files
non_prog_files = fileList(non_prog_path)


# Returns all prog rock files
def get_prog_files():
    return prog_files


# Returns all non prog rock files
def get_non_prog_files() :
    return non_prog_files


In [3]:
import os
import csv
import sys
import math
import cv2
import scipy
import pickle
import librosa
import matplotlib
import numpy as np
import librosa.display
import IPython.display as ipd
matplotlib.use('Agg')
import matplotlib.pyplot as plt
# print(os.getcwd())
# sys.exit(1)
# Import other project files

sys.setrecursionlimit(10000)
prog_files = get_prog_files()
non_prog_files = get_non_prog_files()

print("Number of prog songs",len(prog_files))
print("Number of non prog songs",len(non_prog_files))

all_files = prog_files + non_prog_files
fixed_sr = 44100
min_duration = 0
# -------------------------- Find minimum duration -------------------------------------------------

for i in range(len(all_files)) :
        filename = all_files[i]
        y, sr = librosa.load(filename)
        curr_duration = librosa.get_duration(y=y, sr=sr)
        if i == 0 :
            min_duration = curr_duration
        else :
            min_duration = min(min_duration,curr_duration)
#min_duration = 30#60.041
min_duration=10
min_duration = int(min_duration)
print("min duration ",min_duration)
# -------------------------- Feature extraction ----------------------------------------------------
file = open('training_features.csv', 'w', newline='')
header = 'filename genre chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'

for i in range(1, 21):
    header += f' mfcc{i}'

header = header.split()

# Create file to write error logs
error_logs = open("error_logs.txt","w")
error_logs.close()
with file:
    writer = csv.writer(file)
    writer.writerow(header)

genre = 'prog'
count = 0
time_series_length = 30
# Read prog files
for i in range(len(prog_files)) :
        print("prog ",i)
        filename = prog_files[i]
        name = (filename.split("/") )[-1]
        name = name.replace(" ","_")

        try:
            y, sr = librosa.load(filename,sr = fixed_sr)
            time = librosa.get_duration(y=y,sr=sr)
            chunks = []
            if time > min_duration :
                org_y = y
                iter = math.floor(time/min_duration)
                print(iter)

                current_size = time*fixed_sr
                chunk_size = min_duration*fixed_sr
                start = 0#math.floor(iter/3) * chunk_size
                end = chunk_size
                chunk_index = 1
                #iter = math.floor(iter/3)

                while iter !=0 :
                    count += 1
                    chunk = y[start:end]

                    chroma_stft = librosa.feature.chroma_stft(y=chunk, sr=sr)
                    spec_cent = librosa.feature.spectral_centroid(y=chunk, sr=sr)
                    spec_bw = librosa.feature.spectral_bandwidth(y=chunk, sr=sr)
                    rmse = librosa.feature.rms(y=chunk)
                    rolloff = librosa.feature.spectral_rolloff(y=chunk, sr=sr)
                    zcr = 10**10*np.mean(librosa.zero_crossings(org_y)/len(org_y) )
                    mfcc = librosa.feature.mfcc(y=chunk, sr=sr)
                    to_append = f'{"prog"+name+"chunk"+str(chunk_index)} {genre} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {zcr}'

                    # Append all mfcc features i.e., 20 rows
                    for e in mfcc:
                        to_append += f' {np.mean(e)}'

                    file = open('training_features.csv', 'a', newline='')
                    with file:
                        writer = csv.writer(file)
                        writer.writerow(to_append.split())

                    # spect = librosa.feature.melspectrogram(y=chunk, sr=sr,n_fft=2048, hop_length=512)
                    # spect = librosa.power_to_db(spect, ref=np.max)
                    # plt.figure(figsize=(14, 5))
                    # plt.axis('off')
                    # librosa.display.specshow(spect, fmax=8000)
                    # plt.show()

                    chunk_index += 1


                    start = end
                    end = end + chunk_size
                    iter -= 1
                    # if chunk_index >=20 :
                    #     break
                # print("chunk size ",chunk_size)
                # print("current song size ",current_size)
            else :
                count += 1
                y, sr = librosa.load(filename,sr = fixed_sr,duration=min_duration)

                chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
                spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
                spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
                rmse = librosa.feature.rms(y=y)
                rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
                zcr = 10**10*np.mean(librosa.zero_crossings(org_y)/len(org_y) )
                mfcc = librosa.feature.mfcc(y=y, sr=sr)
                to_append = f'{"prog"+name+"chunk1"} {genre} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {zcr}'

                # Append all mfcc features i.e., 20 rows
                for e in mfcc:
                    to_append += f' {np.mean(e)}'

                file = open('training_features.csv', 'a', newline='')
                with file:
                    writer = csv.writer(file)
                    writer.writerow(to_append.split())

                # spect = librosa.feature.melspectrogram(y=y, sr=sr,n_fft=2048, hop_length=512)
                # spect = librosa.power_to_db(spect, ref=np.max)
                # plt.figure(figsize=(14, 5))
                # plt.axis('off')
                # librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='hz')
                # plt.show()



        except Exception as e :
            print("error handled")
            error_logs = open("error_logs.txt","a")
            error_logs.write(filename)
            error_logs.write("\n")
            error_logs.write(str(e))
            error_logs.write("\n")
            error_logs.close()
            continue


genre = 'non_prog'

# Read Non prog files
for i in range(len(non_prog_files)) :
        print("nonprog ",i)
        filename = non_prog_files[i]
        name = (filename.split("/") )[-1]
        name = name.replace(" ","_")


        try:
            y, sr = librosa.load(filename,sr = fixed_sr)
            time = librosa.get_duration(y=y,sr=sr)
            chunks = []
            if time > min_duration :
                org_y = y
                iter = math.floor(time/min_duration)
                print(iter)

                current_size = time*fixed_sr
                chunk_size = min_duration*fixed_sr
                start = 0#math.floor(iter/3) * chunk_size
                end = chunk_size
                chunk_index = 1
                # iter = math.floor(iter/3)

                while iter !=0 :
                    count += 1
                    chunk = y[start:end]

                    # Display Spectrogram
                    # spect = librosa.feature.melspectrogram(y=chunk, sr=sr,n_fft=2048, hop_length=512)
                    # spect = librosa.power_to_db(spect, ref=np.max)
                    # plt.figure(figsize=(14, 5))
                    # plt.axis('off')
                    # plt.show()

                    chroma_stft = librosa.feature.chroma_stft(y=chunk, sr=sr)
                    spec_cent = librosa.feature.spectral_centroid(y=chunk, sr=sr)
                    spec_bw = librosa.feature.spectral_bandwidth(y=chunk, sr=sr)
                    rmse = librosa.feature.rms(y=chunk)
                    rolloff = librosa.feature.spectral_rolloff(y=chunk, sr=sr)
                    zcr = 10**10*np.mean(librosa.zero_crossings(org_y)/len(org_y) )
                    mfcc = librosa.feature.mfcc(y=chunk, sr=sr)
                    to_append = f'{"nonprog"+name+"chunk"+str(chunk_index)} {genre} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {zcr}'

                    # Append all mfcc features i.e., 20 rows
                    for e in mfcc:
                        to_append += f' {np.mean(e)}'

                    file = open('training_features.csv', 'a', newline='')
                    with file:
                        writer = csv.writer(file)
                        writer.writerow(to_append.split())

                    chunk_index += 1

                    # print("start ",start)
                    # print("end ",end)

                    start = end
                    end = end + chunk_size
                    iter -= 1
                    # if chunk_index >=20 :
                    #     break

            else :
                count += 1
                y, sr = librosa.load(filename,sr = fixed_sr,duration=min_duration)

                # Display Spectrogram
                # spect = librosa.feature.melspectrogram(y=y, sr=sr,n_fft=2048, hop_length=512)
                # spect = librosa.power_to_db(spect, ref=np.max)
                # plt.figure(figsize=(14, 5))
                # plt.axis('off')
                # librosa.display.specshow(spect, sr=sr, x_axis='time', y_axis='hz')
                # plt.show()

                chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
                spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
                spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
                rmse = librosa.feature.rms(y=y)
                rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
                zcr = 10**10*np.mean(librosa.zero_crossings(org_y)/len(org_y) )
                mfcc = librosa.feature.mfcc(y=y, sr=sr)
                to_append = f'{"nonprog"+name+"chunk1"} {genre} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {zcr}'

                # Append all mfcc features i.e., 20 rows
                for e in mfcc:
                    to_append += f' {np.mean(e)}'

                file = open('training_features.csv', 'a', newline='')
                with file:
                    writer = csv.writer(file)
                    writer.writerow(to_append.split())


        except Exception as e:
            print("error handled")
            error_logs = open("error_logs.txt","a")
            error_logs.write(filename)
            error_logs.write("\n")
            error_logs.write(str(e))
            error_logs.write("\n")
            error_logs.close()
            continue
print("tot count ",count)



Number of prog songs 142
Number of non prog songs 359


<ipython-input-3-0d10f0db4f6e>:33: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(filename)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
<ipython-input-3-0d10f0db4f6e>:33: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(filename)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
<ipython-input-3-0d10f0db4f6e>:33: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(filename)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:183: FutureWarning:

min duration  10
prog  0
33
prog  1
38
prog  2
102
prog  3
63
prog  4
42
prog  5
39
prog  6
32
prog  7
79
prog  8
77
prog  9
40
prog  10
67
prog  11
25
prog  12
140
prog  13
33
prog  14
11
prog  15
45
prog  16
53
prog  17
45
prog  18
45
prog  19
83
prog  20
100
prog  21
49
prog  22
73
prog  23
74
prog  24
45
prog  25
93
prog  26
106
prog  27
40
prog  28
60
prog  29
81
prog  30
124
prog  31
123
prog  32
43
prog  33
46
prog  34
123
prog  35
159


/usr/local/lib/python3.10/dist-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


prog  36
144
prog  37
51
prog  38
41
prog  39
55
prog  40
262
prog  41
114
prog  42
28
prog  43
85
prog  44
99
prog  45
45
prog  46
68
prog  47
58
prog  48
50
prog  49
96
prog  50
72
prog  51
55
prog  52
40
prog  53
64
prog  54
34
prog  55
138
prog  56
65
prog  57
62
prog  58
35
prog  59
44
prog  60
52
prog  61
105
prog  62
24
prog  63
50
prog  64
40
prog  65
21
prog  66
51
prog  67
33
prog  68
51
prog  69
182
prog  70
121
prog  71
55
prog  72
148
prog  73
56
prog  74
43
prog  75
28
prog  76
58
prog  77
69
prog  78
131
prog  79
75
prog  80
136
prog  81
53
prog  82
58
prog  83
37
prog  84
94
prog  85
101
prog  86
137
prog  87
136
prog  88
57
prog  89
62
prog  90
70
prog  91
55
prog  92
63
prog  93
41
prog  94
61
prog  95
25
prog  96
52
prog  97
52
prog  98
44
prog  99
32
prog  100
103
prog  101
57
prog  102
22
prog  103
81
prog  104
44
prog  105
43
prog  106
56
prog  107
124
prog  108
68
prog  109
139
prog  110
76
prog  111
180
prog  112
44
prog  113
51
prog  114
25
prog  115
72
prog  1

...

29
nonprog  209
12
nonprog  210
32
nonprog  211
28
nonprog  212
22
nonprog  213
29
nonprog  214
25
nonprog  215
15
nonprog  216
33
nonprog  217
23
nonprog  218
31
nonprog  219
52
nonprog  220
27
nonprog  221
41
nonprog  222
14
nonprog  223
19
nonprog  224
35
nonprog  225
21
nonprog  226
21
nonprog  227
24
nonprog  228
23
nonprog  229
23
nonprog  230
56
nonprog  231
50
nonprog  232
29
nonprog  233
61
nonprog  234
15
nonprog  235
18
nonprog  236
19
nonprog  237
23
nonprog  238
24
nonprog  239
23
nonprog  240
17
nonprog  241
20
nonprog  242
89
nonprog  243
21
nonprog  244
26
nonprog  245
25
nonprog  246
27
nonprog  247
29
nonprog  248
37
nonprog  249
58
nonprog  250
22
nonprog  251
26
nonprog  252
15
nonprog  253
17
nonprog  254
22
nonprog  255
36
nonprog  256
26
nonprog  257
20
nonprog  258
38
nonprog  259
21
nonprog  260
21
nonprog  261
24
nonprog  262
19
nonprog  263
44
nonprog  264
24
nonprog  265
15
nonprog  266
25
nonprog  267
56
nonprog  268
28
nonprog  269
24
nonprog  270
79
nonpr

/usr/local/lib/python3.10/dist-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


nonprog  274
25
nonprog  275
28
nonprog  276
21
nonprog  277
26
nonprog  278
19
nonprog  279
38
nonprog  280
21
nonprog  281


<ipython-input-3-0d10f0db4f6e>:177: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(filename,sr = fixed_sr)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


13
nonprog  282
33
nonprog  283
36
nonprog  284
22
nonprog  285
18
nonprog  286
35
nonprog  287
3
nonprog  288
25
nonprog  289
13
nonprog  290
13
nonprog  291
23
nonprog  292
20
nonprog  293
34
nonprog  294
33
nonprog  295
42
nonprog  296
16
nonprog  297
21
nonprog  298
20
nonprog  299
18
nonprog  300
51
nonprog  301
20
nonprog  302
14
nonprog  303
39
nonprog  304
33
nonprog  305
21
nonprog  306
23
nonprog  307
21
nonprog  308
15
nonprog  309
23
nonprog  310
23
nonprog  311
26
nonprog  312
22
nonprog  313
19
nonprog  314
17
nonprog  315
21
nonprog  316
30
nonprog  317
25
nonprog  318
33
nonprog  319
20
nonprog  320
19
nonprog  321
22
nonprog  322
18
nonprog  323
22
nonprog  324
28
nonprog  325
23
nonprog  326
30
nonprog  327
24
nonprog  328
12
nonprog  329
21
nonprog  330
18
nonprog  331
35
nonprog  332
20
nonprog  333
17
nonprog  334
16
nonprog  335
16
nonprog  336
25
nonprog  337
16
nonprog  338
19
nonprog  339
18
nonprog  340
22
nonprog  341
25
nonprog  342
21
nonprog  343
17
nonpro

In [2]:
import os
import sys
import librosa
import numpy as np

current_working_directory = os.getcwd() # get current working directory

# Address of root directory

prog_path = "/content/drive/MyDrive/Music-Genre-Classification-master-20240427T160246Z-001/Music-Genre-Classification-master/Test_set/Other"
non_prog_path =""


# Returns all audio files(.mp3, .avi, .wav) in the directory : path
def fileList(path) :
    matches = []
    for root, _, filenames in os.walk(path,topdown=True):
        for filename in filenames:
            if filename.endswith(('.mp3', '.wav', '.avi','.flac','.m4a','.ogg')):
                if matches.count(filename) == 0 :
                    matches.append(os.path.join(root, filename))

    return matches


# prog_files contains all prog_rock files
prog_files = fileList(prog_path)

# non_prog_files contains all non prog_rock files
non_prog_files = fileList(non_prog_path)



# Returns all prog rock files
def get_prog_files():
    return prog_files


# Returns all non prog rock files
def get_non_prog_files() :
    return non_prog_files

# Returns all djent files




In [3]:
import os
import csv
import sys
import math
import cv2
import scipy
import pickle
import librosa
import matplotlib
import numpy as np
import librosa.display
import IPython.display as ipd
matplotlib.use('Agg')
import matplotlib.pyplot as plt
# print(os.getcwd())
# sys.exit(1)
# Import other project files

sys.setrecursionlimit(10000)
prog_files = get_prog_files()
non_prog_files = get_non_prog_files()

print("Number of prog songs",len(prog_files))
print("Number of non prog songs",len(non_prog_files))

all_files = prog_files + non_prog_files
fixed_sr = 44100
min_duration = 0
# -------------------------- Find minimum duration -------------------------------------------------
'''
for i in range(len(all_files)) :
        filename = all_files[i]
        y, sr = librosa.load(filename,sr = fixed_sr)
        curr_duration = librosa.get_duration(y=y, sr=sr)
        if i == 0 :
            min_duration = curr_duration
        else :
            min_duration = min(min_duration,curr_duration)
'''
min_duration = 10
min_duration = int(min_duration)
print("min duration ",min_duration)
# -------------------------- Feature extraction ----------------------------------------------------
file = open('test_features.csv', 'w', newline='')
header = 'filename genre chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'

for i in range(1, 21):
    header += f' mfcc{i}'

header = header.split()

# Create file to write error logs
error_logs = open("error_logs.txt","w")
error_logs.close()
with file:
    writer = csv.writer(file)
    writer.writerow(header)

genre = 'prog'
count = 0
time_series_length = 30
# Read prog files
for i in range(len(prog_files)) :
        print("prog ",i)
        filename = prog_files[i]
        name = (filename.split("/") )[-1]
        name = name.replace(" ","_")

        try:
            y, sr = librosa.load(filename,sr = fixed_sr)
            time = librosa.get_duration(y=y,sr=sr)
            chunks = []
            if time > min_duration :
                org_y = y
                iter = math.floor(time/min_duration)
                print(iter)

                current_size = time*fixed_sr
                chunk_size = min_duration*fixed_sr
                start = 0#math.floor(iter/3) * chunk_size
                end = chunk_size
                chunk_index = 1
                # iter = math.floor(iter/3)

                while iter !=0 :
                    count += 1
                    chunk = y[start:end]

                    chroma_stft = librosa.feature.chroma_stft(y=chunk, sr=sr)
                    spec_cent = librosa.feature.spectral_centroid(y=chunk, sr=sr)
                    spec_bw = librosa.feature.spectral_bandwidth(y=chunk, sr=sr)
                    rmse = librosa.feature.rms(y=chunk)
                    rolloff = librosa.feature.spectral_rolloff(y=chunk, sr=sr)
                    zcr = 10**10*np.mean(librosa.zero_crossings(org_y)/len(org_y) )

                    mfcc = librosa.feature.mfcc(y=chunk, sr=sr)
                    if np.mean(chroma_stft) == 0 :
                        break

                    to_append = f'{"prog"+name+"chunk"+str(chunk_index)} {genre} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {zcr}'

                    # Append all mfcc features i.e., 20 rows
                    for e in mfcc:
                        to_append += f' {np.mean(e)}'

                    file = open('test_features.csv', 'a', newline='')
                    with file:
                        writer = csv.writer(file)
                        writer.writerow(to_append.split())


                    chunk_index += 1


                    start = end
                    end = end + chunk_size
                    iter -= 1
                    # if chunk_index >=20 :
                    #     break
                # print("chunk size ",chunk_size)
                # print("current song size ",current_size)
            else :
                count += 1
                y, sr = librosa.load(filename,sr = fixed_sr,duration=min_duration)

                chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
                spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
                spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
                rmse = librosa.feature.rms(y=y)
                rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
                zcr = 10**10*np.mean(librosa.zero_crossings(org_y)/len(org_y) )
                mfcc = librosa.feature.mfcc(y=y, sr=sr)
                to_append = f'{"prog"+name+"chunk1"} {genre} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {zcr}'

                # Append all mfcc features i.e., 20 rows
                for e in mfcc:
                    to_append += f' {np.mean(e)}'

                file = open('test_features.csv', 'a', newline='')
                with file:
                    writer = csv.writer(file)
                    writer.writerow(to_append.split())





        except Exception as e :
            print("error handled")
            error_logs = open("error_logs.txt","a")
            error_logs.write(filename)
            error_logs.write("\n")
            error_logs.write(str(e))
            error_logs.write("\n")
            error_logs.close()
            continue


genre = 'non_prog'

# Read Non prog files
for i in range(len(non_prog_files)) :
        print("nonprog ",i)
        filename = non_prog_files[i]
        name = (filename.split("/") )[-1]
        name = name.replace(" ","_")

        # print("target ",duration*fixed_sr)
        try:
            y, sr = librosa.load(filename,sr = fixed_sr)
            time = librosa.get_duration(y=y,sr=sr)
            chunks = []
            if time > min_duration :
                org_y = y
                iter = math.floor(time/min_duration)
                print(iter)

                current_size = time*fixed_sr
                chunk_size = min_duration*fixed_sr
                start = 0#math.floor(iter/3) * chunk_size
                end = chunk_size
                chunk_index = 1
                # iter = math.floor(iter/3)

                while iter !=0 :
                    count += 1
                    chunk = y[start:end]

                    chroma_stft = librosa.feature.chroma_stft(y=chunk, sr=sr)
                    spec_cent = librosa.feature.spectral_centroid(y=chunk, sr=sr)
                    spec_bw = librosa.feature.spectral_bandwidth(y=chunk, sr=sr)
                    rmse = librosa.feature.rms(y=chunk)
                    rolloff = librosa.feature.spectral_rolloff(y=chunk, sr=sr)
                    zcr = 10**10*np.mean(librosa.zero_crossings(org_y)/len(org_y) )
                    mfcc = librosa.feature.mfcc(y=chunk, sr=sr)
                    if np.mean(chroma_stft) == 0 :
                        break

                    to_append = f'{"nonprog"+name+"chunk"+str(chunk_index)} {genre} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {zcr}'

                    # Append all mfcc features i.e., 20 rows
                    for e in mfcc:
                        to_append += f' {np.mean(e)}'

                    file = open('test_features.csv', 'a', newline='')
                    with file:
                        writer = csv.writer(file)
                        writer.writerow(to_append.split())



                    chunk_index += 1



                    start = end
                    end = end + chunk_size
                    iter -= 1

            else :
                count += 1
                y, sr = librosa.load(filename,sr = fixed_sr,duration=min_duration)


                chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
                spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
                spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
                rmse = librosa.feature.rms(y=y)
                rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
                zcr = 10**10*np.mean(librosa.zero_crossings(org_y)/len(org_y) )
                mfcc = librosa.feature.mfcc(y=y, sr=sr)
                to_append = f'{"nonprog"+name+"chunk1"} {genre} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {zcr}'

                # Append all mfcc features i.e., 20 rows
                for e in mfcc:
                    to_append += f' {np.mean(e)}'

                file = open('test_features.csv', 'a', newline='')
                with file:
                    writer = csv.writer(file)
                    writer.writerow(to_append.split())



        except Exception as e:
            print("error handled")
            error_logs = open("error_logs.txt","a")
            error_logs.write(filename)
            error_logs.write("\n")
            error_logs.write(str(e))
            error_logs.write("\n")
            error_logs.close()
            continue
print("tot count ",count)


# Display Spectrogram
# STFT converts signal such that we can know the amplitude of given frequency at a given time.
# Using STFT we can determine the amplitude of various frequencies playing at a given time of an audio signal
'''
    X = librosa.stft(x)
    Xdb = librosa.amplitude_to_db(abs(X))
    plt.figure(figsize=(14, 5))
    librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='hz')
    plt.colorbar()
    plt.show()
'''


Number of prog songs 22
Number of non prog songs 0
min duration  10
prog  0
26
prog  1
31
prog  2
19
prog  3
15
prog  4
37


/usr/local/lib/python3.10/dist-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


prog  5
36
prog  6
28
prog  7
30
prog  8
30
prog  9
25
prog  10
20
prog  11
23
prog  12
25
prog  13
24
prog  14
43
prog  15
37
prog  16
24
prog  17
25
prog  18
26
prog  19
38
prog  20
30
prog  21
43
tot count  631


"\n    X = librosa.stft(x)\n    Xdb = librosa.amplitude_to_db(abs(X))\n    plt.figure(figsize=(14, 5))\n    librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='hz')\n    plt.colorbar()\n    plt.show()\n"

In [4]:
import os
import sys
import librosa
import numpy as np

current_working_directory = os.getcwd() # get current working directory


djent_path ="/content/drive/MyDrive/Music-Genre-Classification-master-20240427T160246Z-001/Music-Genre-Classification-master/Test_set/Other"

# Returns all audio files(.mp3, .avi, .wav) in the directory : path
def fileList(path) :
    matches = []
    for root, _, filenames in os.walk(path,topdown=True):
        for filename in filenames:
            if filename.endswith(('.mp3', '.wav', '.avi','.flac','.m4a','.ogg')):
                if matches.count(filename) == 0 :
                    matches.append(os.path.join(root, filename))

    return matches



djent_files = fileList(djent_path)


# Returns all djent files
def get_djent_files() :
    return djent_files

In [7]:
import os
import csv
import sys
import math
import cv2
import scipy
import pickle
import librosa
import matplotlib
import numpy as np
import librosa.display
import IPython.display as ipd
matplotlib.use('Agg')
import matplotlib.pyplot as plt
# print(os.getcwd())
# sys.exit(1)
# Import other project files
sys.setrecursionlimit(10000)
djent_files = get_djent_files()

print("Number of djent songs",len(djent_files))

all_files = djent_files
fixed_sr = 44100
min_duration = 0
# -------------------------- Find minimum duration -------------------------------------------------

min_duration = 10#60.041
min_duration = int(min_duration)
print("min duration ",min_duration)
# -------------------------- Feature extraction ----------------------------------------------------
file = open('test_djent_features.csv', 'w', newline='')
header = 'filename genre chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'

for i in range(1, 21):
    header += f' mfcc{i}'

header = header.split()

# Create file to write error logs
error_logs = open("error_logs.txt","w")
error_logs.close()
with file:
    writer = csv.writer(file)
    writer.writerow(header)

genre = 'djent'
count = 0
time_series_length = 30
# Read prog files
for i in range(len(all_files)) :
        print("djent ",i)
        filename = all_files[i]
        name = (filename.split("/") )[-1]
        name = name.replace(" ","_")

        try:
            y, sr = librosa.load(filename,sr = fixed_sr)
            time = librosa.get_duration(y=y,sr=sr)
            chunks = []
            if time > min_duration :
                org_y = y
                iter = math.floor(time/min_duration)
                print(iter)

                current_size = time*fixed_sr
                chunk_size = min_duration*fixed_sr
                start = 0#math.floor(iter/3) * chunk_size
                end = chunk_size
                chunk_index = 1
                # iter = math.floor(iter/3)

                while iter !=0 :
                    count += 1
                    chunk = y[start:end]

                    chroma_stft = librosa.feature.chroma_stft(y=chunk, sr=sr)
                    spec_cent = librosa.feature.spectral_centroid(y=chunk, sr=sr)
                    spec_bw = librosa.feature.spectral_bandwidth(y=chunk, sr=sr)
                    rmse = librosa.feature.rms(y=chunk)
                    rolloff = librosa.feature.spectral_rolloff(y=chunk, sr=sr)
                    zcr = 10**10*np.mean(librosa.zero_crossings(org_y)/len(org_y) )

                    mfcc = librosa.feature.mfcc(y=chunk, sr=sr)
                    if np.mean(chroma_stft) == 0 :
                        break
                    to_append = f'{"djent"+name+"chunk"+str(chunk_index)} {genre} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {zcr}'

                    # Append all mfcc features i.e., 20 rows
                    for e in mfcc:
                        to_append += f' {np.mean(e)}'

                    file = open('test_djent_features.csv', 'a', newline='')
                    with file:
                        writer = csv.writer(file)
                        writer.writerow(to_append.split())

                    # spect = librosa.feature.melspectrogram(y=chunk, sr=sr,n_fft=2048, hop_length=512)
                    # spect = librosa.power_to_db(spect, ref=np.max)
                    # plt.figure(figsize=(14, 5))
                    # plt.axis('off')
                    # librosa.display.specshow(spect, fmax=8000)

                    # plt.savefig("outputprog.png",bbox_inches='tight',transparent=True,pad_inches=0)
                    # im = cv2.imread("outputprog.png")
                    # plt.clf()
                    # plt.cla()
                    # plt.close()

                    # np.savez_compressed("/Users/richadutt/Documents/ranga/mfcc_validation_set/prog-"+name+"chunk"+str(chunk_index),im,im)
                    chunk_index += 1


                    start = end
                    end = end + chunk_size
                    iter -= 1
                    # if chunk_index >=20 :
                    #     break
                # print("chunk size ",chunk_size)
                # print("current song size ",current_size)
            else :
                count += 1
                y, sr = librosa.load(filename,sr = fixed_sr,duration=min_duration)

                chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
                spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
                spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
                rmse = librosa.feature.rms(y=y)
                rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
                zcr = 10**10*np.mean(librosa.zero_crossings(org_y)/len(org_y) )
                mfcc = librosa.feature.mfcc(y=y, sr=sr)
                to_append = f'{"djent"+name+"chunk1"} {genre} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {zcr}'

                # Append all mfcc features i.e., 20 rows
                for e in mfcc:
                    to_append += f' {np.mean(e)}'

                file = open('test_djent_features.csv', 'a', newline='')
                with file:
                    writer = csv.writer(file)
                    writer.writerow(to_append.split())

                # spect = librosa.feature.melspectrogram(y=y, sr=sr,n_fft=2048, hop_length=512)
                # spect = librosa.power_to_db(spect, ref=np.max)

                # plt.figure(figsize=(14, 5))
                # plt.axis('off')
                # X = librosa.stft(y)
                # Xdb = librosa.amplitude_to_db(abs(X))
                # librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='hz')

                # plt.savefig("outputprog.png",bbox_inches='tight',transparent=True,pad_inches=0)
                # im = cv2.imread("outputprog.png")
                # plt.clf()
                # plt.cla()
                # plt.close()

                # np.savez_compressed("/Users/richadutt/Documents/ranga/mfcc_validation_set/prog-"+name+"chunk1",im,im)

            # print(i," ---->  ",y.shape)



        except Exception as e :
            print("error handled")
            error_logs = open("error_logs.txt","a")
            error_logs.write(filename)
            error_logs.write("\n")
            error_logs.write(str(e))
            error_logs.write("\n")
            error_logs.close()
            continue

Number of djent songs 22
min duration  10
djent  0
26
djent  1
31
djent  2
19
djent  3
15
djent  4
37
djent  5
36
djent  6
28
djent  7
30
djent  8
30
djent  9
25
djent  10
20
djent  11
23
djent  12
25
djent  13
24
djent  14
43
djent  15
37
djent  16
24
djent  17
25
djent  18
26
djent  19
38
djent  20
30
djent  21
43
